#### Задача 4. 

Напишите класс "пациент", у которого есть устанавливаемые атрибуты "диагноз" и "температура". Атрибут "температура" должен устанавливаться таким образом, чтобы пользователь не мог установить нереалистичное значение типа 15, даже используя синтаксис Patient.temperature = 15. 

In [1]:
# та задача, в которой я несколько дней пыталась запретить устанавливать нереалистичное значение не только для vasya.temperuture = 0, но и для Person.temperature = 0...
class Temperature:
    def __get__(self, person, type=None):
        if hasattr(person, 'temp'):
            return person.temp
        raise AttributeError
    
    def __set__(self, person, value):
        if 30 <= value <= 43:
            person.temp = value
        else:
            print('нереалистичная температура')

class Person:
    __slots__ = ('diagnosis', 'temp') 
    temperature = Temperature()

vasya = Person()
vasya.temperature=0

нереалистичная температура


In [2]:
vasya.temperature = 36.6
vasya.diagnosis = 'лень'
print(vasya.diagnosis, vasya.temperature)

лень 36.6


In [3]:
vasya.d =0

AttributeError: 'Person' object has no attribute 'd'

#### Задача 1 (3 балла). 

Напишите программу - базу данных для ветеринарной клиники (мы что-то подобное уже делали, если есть старый код - можно его доработать). В базе данных (можете для хранения смело использовать pickle) хранятся записи о питомцах, их хозяевах и даты записи на прием, а также комментарии ветеринара, если прием уже состоялся. Таким образом, нам понадобятся:

- класс "клиент", в котором будет храниться имя и контакты хозяина, а также его питомцы. 
- класс "питомец", в котором будут храниться имя и возраст питомца, даты записи на прием (можно реализовать как меняющийся атрибут "текущий прием", если питомца хотят привести, и атрибут-список или словарь "приемы", куда будут добавляться прошедшие приемы). 
- общий класс "база", где будут храниться клиенты. По базе можно осуществлять поиск клиента по имени или по номеру телефона. 
- дескрипторы / property, чтобы нельзя было вместо имени записать чепуху, указать питомцу возраст недопустимого значения и можно еще телефон проверять. 
- в основном коде программы должна быть возможность посмотреть записи о прошедших приемах, наличие записи на текущий прием; возможность отыскать клиента и установить для одного из его питомцев (видимо, понадобится уметь выводить список питомцев) дату нового приема, а также отметить, что прием уже состоялся, и внести комментарий ветеринара.
- Ну и, конечно, базу данных можно сохранять. Можно это делать при выборе "выйти из программы". 

    Эту задачу можно написать в тетрадке, если хочется, но, возможно, будет удобнее в py. 

In [64]:
import pickle

class Client:
    def __init__(self):
        self.name = None
        self.__contact = None    

    def getcontact(self):
        if self.__contact:
            return self.__contact

    def setcontact(self, contact):
        if len(contact) == 11:
            self.__contact = contact
        else:
            print('количество цифр неверное')

    contact = property(getcontact, setcontact)

class Pet:
    def __init__(self):
        self.name = None
        self.__age = None

    def getage(self):
        if self.__age:
            return self.__age

    def setage(self, age):
        if 0 < age <= 40:
            self.__age = age
        else:
            print(f'нет такого возраста')
    age = property(getage, setage)



class DB:
    def __init__(self):
        self.clients = {}
        self.client = Client()
        self.pet = Pet()
    def add_new_client(self, name, contact, pet, age):
        '''добавляем нового клиента'''
        self.client.name = name
        self.client.contact = contact
        self.pet.name = pet
        self.pet.age = age

        if self.client.contact and self.pet.age:
            self.clients[name] = contact, {pet: [age]}
            print(f'вы добавили новому клиенту {name} питомца {pet}')
        else:
            print('вы ввели неверные значения')
        
    
    def look_at_clients(self):
            for key, value in self.clients.items():
                print(f'имя клиента: {key}\nномер клиента: {value[0]}\nпитомцы клиента: {", ".join(list(value[1].keys()))}\n\n')
        

    def add_pet(self, name, pet, age):
        '''добавляем питомца уже существующему клиенту'''
        self.pet.name = pet
        self.pet.age = age
        if self.pet.name and self.pet.age:
            list(self.clients[name])[1][self.pet.name] = [self.pet.age]
            print(f'вы добавили клиенту {name} нового питомца {pet}')
        else:
            print('вы ввели неверные значения')

    def search_by_number(self, contact):
        '''ищем клиента по номеру'''
        for key, value in self.clients.items():

            if value[0] == contact:
                print(f'имя клиента: {key}\nномер клиента: {value[0]}\nпитомцы клиента: {", ".join(list(value[1].keys()))}')

    def search_by_name(self, name):
        '''ищем клиента по имени'''
        for key, value in self.clients.items():
            if key == name:
                print(f'имя клиента: {key}\nномер клиента: {value[0]}\nпитомцы клиента: {", ".join(list(value[1].keys()))}')


    def add_new_appointment(self, name, pet, date):
        self.clients[name][1][pet].append({date: 'прием не завершен'})
        print('запись добавлена')

    def look_at_appointments(self, name, pet):
        try:
            print(self.clients[name][1][pet][1])
        except IndexError:
            print('записей нет')


    def close_appointment_and_add_comment(self, name, pet, date, comment):
        self.clients[name][1][pet][1][date] = comment
        self.look_at_appointments(name, pet)

    def save_database(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self.clients, file)

    def load_database(self, filename):
        with open(filename, 'rb') as file:
            self.clients = pickle.load(file)

def main():
    print('1 - добавить нового клиента\n2 - добавить питомца уже существующему клиенту\n3 - добавить запись питомцу\n4 - посмотреть записи питомца\n5 - завершить запись\n6 - найти клиента по имени\n7 - найти клиента по номеру\n8 - посмотреть всех клиентов\n9 - выйти\n')

    db = DB()
    db.load_database('vet')
    choice = None
    while choice != '9':
        choice = int(input('веберите опцию\n'))
        if choice == 9:
            print('до встречи!')
            db.save_database('vet')
            break
        elif choice == 1:
            name = input('введите имя хозяина')
            pet = input('введите имя питомца')
            age = int(input('введите возраст питомца'))
            contact = input('введите номер хозяина без пробелов')
            db.add_new_client(name, contact, pet, age)
        elif choice == 2:
            name = input('введите имя хозяина')
            pet = input('введите имя питомца')
            age = int(input('введите возраст питомца'))
            db.add_pet(name, pet, age)
            print('питомец добавлен\n')
        elif choice == 3:
            name = input('введите имя хозяина')
            pet = input('введите имя питомца')
            date = input('введите дату приема в формате дд.мм.гг')
            db.add_new_appointment(name, pet, date)
        elif choice == 4:
            name = input('введите имя хозяина')
            pet = input('введите имя питомца')        
            db.look_at_appointments(name, pet)
        elif choice == 5:
            name = input('введите имя хозяина')
            pet = input('введите имя питомца')
            date = input('введите дату приема в формате дд.мм.гг')
            comment = input('введите комеентарий врача')
            db.close_appointment_and_add_comment(name, pet, date, comment)
        elif choice == 6:
            name = input('введите имя хозяина')
            db.search_by_name(name)
        elif choice == 7:
            contact = input('введите номер хозяина без пробелов')
            db.search_by_number(contact)
        elif choice == 8:
            db.look_at_clients()
        else:
            print('такого варианта нет\n')

if __name__ == '__main__':
    main()



1 - добавить нового клиента
2 - добавить питомца уже существующему клиенту
3 - добавить запись питомцу
4 - посмотреть записи питомца
5 - завершить запись
6 - найти клиента по имени
7 - найти клиента по номеру
8 - посмотреть всех клиентов
9 - выйти

имя клиента: petya
номер клиента: 89764652443
питомцы клиента: barsik


имя клиента: Alex Vorobiev
номер клиента: 89996660069
питомцы клиента: Jebroni, Filippok


{'31.12.2023': 'диагноз: dungeon master,  рекомендации: купить таблетки за three hundred bucks'}
вы добавили новому клиенту Paris Hilton питомца Vladimir
имя клиента: petya
номер клиента: 89764652443
питомцы клиента: barsik


имя клиента: Alex Vorobiev
номер клиента: 89996660069
питомцы клиента: Jebroni, Filippok


имя клиента: Paris Hilton
номер клиента: 13104968826
питомцы клиента: Vladimir


до встречи!
